In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [15]:
import sys
sys.path.append(r"..")
import cv2
import torch
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
from pysot.core.config import cfg
from pysot.models.model_builder import ModelBuilder
from pysot.tracker.tracker_builder import build_tracker
from birdbox.tools import process_video, Rect, video_drop_down


In [7]:
CONFIG_FILE = Path(r"..\models\pysot\siamrpn_r50_l234_dwxcorr\config.yaml")
SNAPSHOT_FILE = Path(r"..\models\pysot\siamrpn_r50_l234_dwxcorr\model.pth")

class TrackingProcessor:
    def __init__(self):

        cfg.merge_from_file(CONFIG_FILE)
        cfg.CUDA = torch.cuda.is_available() and cfg.CUDA
        device = torch.device('cuda' if cfg.CUDA else 'cpu')

        model = ModelBuilder()
        model.load_state_dict(torch.load(SNAPSHOT_FILE, map_location=lambda storage, loc: storage.cpu()))
        model.eval().to(device)
        self.tracker = build_tracker(model)
        self.init_rect = None

    def __call__(self, frame):
        if not self.init_rect:
            self.init_rect = cv2.selectROI(VIDEO.stem, frame, False, False)
            self.tracker.init(frame, self.init_rect)
            return

        outputs = self.tracker.track(frame)
        bbox = Rect.from_left_top_width_height(*list(map(int, outputs['bbox'])))
        cv2.rectangle(frame, (bbox.left, bbox.top), (bbox.right, bbox.bottom), (0, 255, 0), 3)

In [16]:
drop_down = video_drop_down()
display(drop_down)

Dropdown(description='Video file:', options=('Blue tit vs chaffinch.mp4', 'Verl red robin in water.mp4', 'Grou…

In [17]:
VIDEO = Path(r"..\videos") / drop_down.value

processor = TrackingProcessor()
process_video(VIDEO, processor, skip=0)